**1. write the dual LP problem** 

**Given primal LP problem:**

\begin{align*}
\text{maximize} \quad & x_1 + 4x_2 + 2x_3 \\
\text{subject to} \quad & 5x_1 + 2x_2 + 2x_3 \leq 145 \\
& 4x_1 + 8x_2 - 8x_3 \leq 260 \\
& x_1 + x_2 + 4x_3 \leq 190 \\
& x \geq 0
\end{align*}

**Dual LP problem:**

\begin{align*}
\text{minimize} \quad & 145y_1 + 260y_2 + 190y_3 \\
\text{subject to} \quad & 5y_1 + 4y_2 + y_3 \geq 1 \\
& 2y_1 + 8y_2 + y_3 \geq 4 \\
& 2y_1 - 8y_2 + 4y_3 \geq 2 \\
& y \geq 0
\end{align*}

**Standard form Dual LP problem:**

\begin{align*}
\text{maximize} \quad & -145y_1 - 260y_2 - 190y_3 \\
\text{subject to} \quad & -5y_1 - 4y_2 - y_3 \leq -1 \\
& -2y_1 - 8y_2 - y_3 \leq -4 \\
& -2y_1 + 8y_2 - 4y_3 \leq -2 \\
& y \geq 0
\end{align*}

**2. verify that  $Q=(x_1,x_2,x_3)=(0,52.5,20)$ is a feasible solution for the primal**

In [1]:
""" I used the architecture of the code from E4basicLPcode.ipynb (Cf. GitHub repo ) 
and modified it to fit our new objective function, constraints, and coefficients. I kept the same solver, and simplified the status to only return the two expected outcomes"""

from ortools.linear_solver import pywraplp

def create_solver():
    # Create the linear solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        raise Exception('SCIP solver not available.')
    return solver

def verify_feasibility_Q():
    solver = create_solver()

    # Variables x1, x2, x3
    x1 = solver.NumVar(0, solver.infinity(), 'x1')
    x2 = solver.NumVar(0, solver.infinity(), 'x2')
    x3 = solver.NumVar(0, solver.infinity(), 'x3')

    # Objective Function: this is a feasibility check, so we can use a dummy objective.
    # Here we just minimize x1 which won't affect the feasibility.
    objective = solver.Objective()
    objective.SetCoefficient(x1, 1)
    objective.SetMinimization()

    # Constraints
    constraint1 = solver.Constraint(-solver.infinity(), 145)
    constraint1.SetCoefficient(x1, 5)
    constraint1.SetCoefficient(x2, 2)
    constraint1.SetCoefficient(x3, 2)

    constraint2 = solver.Constraint(-solver.infinity(), 260)
    constraint2.SetCoefficient(x1, 4)
    constraint2.SetCoefficient(x2, 8)
    constraint2.SetCoefficient(x3, -8)

    constraint3 = solver.Constraint(-solver.infinity(), 190)
    constraint3.SetCoefficient(x1, 1)
    constraint3.SetCoefficient(x2, 1)
    constraint3.SetCoefficient(x3, 4)

    # Set the proposed solution Q as a starting point for the solver
    solver.SetHint([x1, x2, x3], [0, 52.5, 20])

    # Solve the problem
    status = solver.Solve()

    # Check and print if the proposed solution Q is feasible
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print('The solution Q is feasible.')
        print(f'x1 = {x1.solution_value()}, x2 = {x2.solution_value()}, x3 = {x3.solution_value()}')
    else:
        print('The solution Q is not feasible.')

# Call the function to verify the feasibility of Q
verify_feasibility_Q()


The solution Q is feasible.
x1 = 0.0, x2 = 0.0, x3 = 0.0


**3. Use Complementary Slackness to determine a candidate solution to the dual.**

To determine a candidate solution to the dual using Column Generation (CG) and display the simplex tableau at each iteration, I needED to implement a structure that can handle the simplex method and also interact with ortools. Using the structure given to us in class; 
https://notebook.community/infimath/optimization-method/Simplex%20Tableau%20in%20Python
I tried to use this tableau generation method with around my code. 



In [9]:
from __future__ import division
import numpy as np
from fractions import Fraction

class Tableau:
    def __init__(self, obj):
        self.obj = [1] + obj
        self.rows = []
        self.cons = []
        self.no_variables = len(obj)
        self.no_constraints = 0
        self.is_fraction = False # set True to output in fraction

    def add_constraint(self, expression, value):
        self.rows.append([0] + expression)
        self.cons.append(value)
        self.no_constraints += 1
        self.header_tableau = ["Basic"] + ["x"+str(i+1) for i in range(self.no_variables)] \
                                        + ["s"+str(i+1) for i in range(self.no_constraints)] \
                                        + ["Solution"]
        self.basic_variables = ["s"+str(i+1) for i in range(self.no_constraints)]

    def _pivot_column(self):
        lowest = min(self.obj[1:-1])
        if lowest >= 0:
            return -1
        return self.obj.index(lowest)

    def _pivot_row(self, col):
        ratios = []
        for i, row in enumerate(self.rows):
            element = row[col]
            if element <= 0:
                ratios.append(float('inf'))  # Avoid division by zero and negative ratios
            else:
                ratios.append(self.rows[i][-1] / element)
        return np.argmin(ratios)

    def display(self):
        if self.is_fraction:
            # Formatting the output in fraction
            fmt = '{:<8}' + ''.join(['{:>8}'.format("x"+str(i+1)) for i in range(self.no_variables)]) \
                + ''.join(['{:>8}'.format("s"+str(i+1)) for i in range(self.no_constraints)]) + '{:>8}'
            print(fmt.format("Basic", *["x"+str(i+1) for i in range(self.no_variables)] + ["s"+str(i+1) for i in range(self.no_constraints)] + ["Sol."]))

            row_format = '{:<8}' + ''.join(['{:>8}'.format(str(Fraction(x).limit_denominator())) for x in self.obj[1:]])
            print(row_format.format("z", *[str(Fraction(x).limit_denominator()) for x in self.obj[1:]]))

            for i, row in enumerate(self.rows):
                print(row_format.format(self.basic_variables[i], *[str(Fraction(x).limit_denominator()) for x in row[1:]]))
        else:
            # Formatting the output in float with 2 decimal places
            fmt = '{:<8}' + ''.join(['{:>8}'.format("x"+str(i+1)) for i in range(self.no_variables)]) \
                + ''.join(['{:>8}'.format("s"+str(i+1)) for i in range(self.no_constraints)]) + '{:>8}'
            print(fmt.format("Basic", *["x"+str(i+1) for i in range(self.no_variables)] + ["s"+str(i+1) for i in range(self.no_constraints)] + ["Sol."]))

            row_format = '{:<8}' + ''.join(['{:>8.2f}'.format(x) for x in self.obj[1:]])
            print(row_format.format("z", *[f'{x:.2f}' for x in self.obj[1:]]))

            for i, row in enumerate(self.rows):
                print(row_format.format(self.basic_variables[i], *[f'{x:.2f}' for x in row[1:]]))


    def _pivot(self, row, col):
        pivot_element = self.rows[row][col]
        self.rows[row] = [x / pivot_element for x in self.rows[row]]
        for r in range(len(self.rows)):
            if r != row:
                self.rows[r] = [orig - factor * updated for orig, factor, updated in zip(self.rows[r], self.rows[row], [self.rows[r][col]]*len(self.rows[row]))]
        self.obj = [z - factor * updated for z, factor, updated in zip(self.obj, self.rows[row], [self.obj[col]]*len(self.obj))]

    def _check(self):
        return min(self.obj[1:-1]) >= 0

    def solve(self):
        # Append slack variables and setup tableau
        for i in range(len(self.rows)):
            identity = [0]*self.no_constraints
            identity[i] = 1
            self.rows[i] += identity + [self.cons[i]]
            self.rows[i] = np.array(self.rows[i], dtype=float)
        self.obj = np.array(self.obj + [0]*self.no_constraints, dtype=float)

        self.display()
        while not self._check():
            col = self._pivot_column()
            if col == -1:
                break
            row = self._pivot_row(col)
            self._pivot(row, col)
            print('\nEntering Variable: ', self.header_tableau[col])
            print('Leaving Variable: ', self.basic_variables[row], '\n')
            self.basic_variables[row] = self.header_tableau[col]
            self.display()

if __name__ == '__main__':
    t = Tableau([-2, -3, -2])
    t.add_constraint


\begin{align*}
\text{maximize} \quad & x_1 + 4x_2 + 2x_3 \\
\text{subject to} \quad & 5x_1 + 2x_2 + 2x_3 \leq 145 \\
& 4x_1 + 8x_2 - 8x_3 \leq 260 \\
& x_1 + x_2 + 4x_3 \leq 190 \\
& x \geq 0
\end{align*}

In [10]:
# Solving the dual LP problem and displaying the tableau


# Then, we instantiate the Tableau class for the dual problem
# Recall the dual problem's objective function and constraints

# Since this is a maximization problem, the objective coefficients should be negated for the Tableau
t = Tableau([-1, -4, -2])  # The negated coefficients of the primal objective function

# Add the constraints of the dual, which are the transposed coefficients of the primal constraints with their signs reversed
t.add_constraint([-5, -4, -1], 145)  # The negated coefficients of the first primal constraint
t.add_constraint([-2, -8, -1], 260)  # The negated coefficients of the second primal constraint
t.add_constraint([-2, 8, -4], 190)   # The negated coefficients of the third primal constraint

t.is_fraction = True
t.solve()

Basic         x1      x2      x3      s1      s2      s3      x1
z             -1      -4      -2       0       0       0
s1            -1      -4      -2       0       0       0
s2            -1      -4      -2       0       0       0
s3            -1      -4      -2       0       0       0


AttributeError: 'numpy.ndarray' object has no attribute 'index'

Still trying out the OR tools code to see if I can obtain the right solutions for y1, y2 and y3. (without displaying the tableau for each iteration)

In [11]:
# Using ortools to solve the dual LP problem

from ortools.linear_solver import pywraplp

def create_solver():
    # Create the linear solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        raise Exception('SCIP solver not available.')
    return solver

def solve_dual_with_ortools():
    solver = create_solver()

    # Variables for the dual problem
    y1 = solver.NumVar(0, solver.infinity(), 'y1')
    y2 = solver.NumVar(0, solver.infinity(), 'y2')
    y3 = solver.NumVar(0, solver.infinity(), 'y3')

    # Dual LP objective: Minimize 145y1 + 260y2 + 190y3
    objective = solver.Objective()
    objective.SetCoefficient(y1, 145)
    objective.SetCoefficient(y2, 260)
    objective.SetCoefficient(y3, 190)
    objective.SetMinimization()

        # Dual constraints (corresponding to the primal variables)
    constraint1 = solver.Constraint(1, solver.infinity())
    constraint1.SetCoefficient(y1, 5)
    constraint1.SetCoefficient(y2, 4)
    constraint1.SetCoefficient(y3, 1)

    constraint2 = solver.Constraint(4, solver.infinity())
    constraint2.SetCoefficient(y1, 2)
    constraint2.SetCoefficient(y2, 8)
    constraint2.SetCoefficient(y3, 1)

    constraint3 = solver.Constraint(2, solver.infinity())
    constraint3.SetCoefficient(y1, 2)
    constraint3.SetCoefficient(y2, -8)
    constraint3.SetCoefficient(y3, 4)

    # Solve the problem
    status = solver.Solve()

    # Check and print the solution or status
    if status == pywraplp.Solver.OPTIMAL:
        print('Optimal Solution Found:')
        print('y1 =', y1.solution_value())
        print('y2 =', y2.solution_value())
        print('y3 =', y3.solution_value())
        print('Objective value =', objective.Value())
    elif status == pywraplp.Solver.INFEASIBLE:
        print('No feasible solution found. The problem is infeasible.')
    elif status == pywraplp.Solver.UNBOUNDED:
        print('The problem is unbounded.')
    else:
        print(f'Solver status is {status}: Problem could not be solved due to other reasons.')

# Call the function to solve the dual with ortools
solve_dual_with_ortools()



Optimal Solution Found:
y1 = 1.5
y2 = 0.12499999999999994
y3 = 0.0
Objective value = 250.00000000000003


I modified the code I was given to approach the dual problem using the simplex method. By setting up a tableau with objective and constraint coefficients, I aimed to iteratively find the optimal solution. Initially, the presence of negative coefficients in the 'Z' row indicated the potential for a better solution. Through the simplex algorithm, I identified the most negative coefficient for the entering variable and performed a minimum ratio test to find the pivot row—avoiding division by zero or negative ratios.

After determining the pivot element, I normalized the pivot row, turning the pivot column into a unit vector, thus bringing the chosen variable into the basis. The tableau was then updated, subtracting multiples of the pivot row from all other rows to zero out the column above the pivot element, ensuring the constraints remained consistent.

The process continued until no more improvements were possible, indicated by non-negative 'Z' coefficients. The optimal 'y' values, representing shadow prices for the primal constraints. This demonstrated a systematic approach to deducing the dual problem's solution using the simplex method.

In [17]:
# Import required libraries
from fractions import Fraction
from numpy import argmin, set_printoptions, zeros, inf

set_printoptions(precision=3, suppress=True)  # Nicer display of numpy arrays

# Define the Tableau class
class Tableau:
    def __init__(self, obj, constraints, b_values):
        self.obj = [1] + obj  # The objective row (first row in tableau)
        self.constraints = constraints  # Coefficients of constraints
        self.b_values = b_values  # Right-hand side values
        self.num_vars = len(obj)  # Number of decision variables
        self.num_slacks = len(b_values)  # Number of slack variables (equal to number of constraints)
        
        # Creating the initial tableau with the objective and constraints
        self.tableau = zeros((1 + self.num_slacks, 1 + self.num_vars + self.num_slacks + 1))
        self.tableau[0, 1:1+self.num_vars] = obj
        for i, (constraint, b_value) in enumerate(zip(constraints, b_values)):
            self.tableau[i+1, 1:1+self.num_vars] = constraint  # Set constraint coefficients
            self.tableau[i+1, -1] = b_value  # Set the b_value
            self.tableau[i+1, 1+self.num_vars+i] = 1  # Set slack variable coefficient

    def display(self):
        # Display the current state of the simplex tableau
        print("Current Tableau:")
        for row in self.tableau:
            print(' | '.join(str(Fraction(str(x)).limit_denominator()) for x in row))
        print()

    def pivot(self):
        # Find the pivot column (most negative value in objective row)
        pivot_col = argmin(self.tableau[0, 1:-1]) + 1  # +1 to correct the index
        if self.tableau[0, pivot_col] >= 0:  # No negative value in objective row means optimal solution reached
            return False
        
        # Minimum ratio test, excluding non-positive entries
        ratios = [self.tableau[i, -1] / self.tableau[i, pivot_col] if self.tableau[i, pivot_col] > 0 else inf for i in range(1, len(self.tableau))]
        pivot_row = argmin(ratios) + 1  # +1 to adjust for objective row

        # If all entries in pivot column are non-positive
        if all(self.tableau[i, pivot_col] <= 0 for i in range(1, len(self.tableau))):
            print("Problem is unbounded.")
            return False

        # Perform the pivot operation
        pivot_val = self.tableau[pivot_row, pivot_col]
        self.tableau[pivot_row, :] /= pivot_val
        for i in range(len(self.tableau)):
            if i != pivot_row:
                self.tableau[i, :] -= self.tableau[i, pivot_col] * self.tableau[pivot_row, :]

        return True  # Indicate successful pivot

    def solve(self):
        # Solve the tableau using the Simplex algorithm
        self.display()  # Display the initial tableau
        while min(self.tableau[0, 1:-1]) < 0:  # While negative coefficients in objective row
            if not self.pivot():  # Perform a pivot operation
                break  # Stop if no valid pivot or problem is unbounded
            self.display()  # Display the tableau after each pivot

# Example use of the Tableau class
dual_obj = [-1, -4, -2]
dual_constraints = [
    [-5, -4, -1],
    [-2, -8, 1],
    [-2, 8, -4]
]
dual_b_values = [145, 260, 190]

tableau = Tableau(dual_obj, dual_constraints, dual_b_values)
tableau.solve()


Current Tableau:
0 | -1 | -4 | -2 | 0 | 0 | 0 | 0
0 | -5 | -4 | -1 | 1 | 0 | 0 | 145
0 | -2 | -8 | 1 | 0 | 1 | 0 | 260
0 | -2 | 8 | -4 | 0 | 0 | 1 | 190

Current Tableau:
0 | -2 | 0 | -4 | 0 | 0 | 1/2 | 95
0 | -6 | 0 | -3 | 1 | 0 | 1/2 | 240
0 | -4 | 0 | -3 | 0 | 1 | 1 | 450
0 | -1/4 | 1 | -1/2 | 0 | 0 | 1/8 | 95/4

Problem is unbounded.


I had run my adapted simplex algorithm code to solve the dual problem. Initially, the tableau displayed negative coefficients in the objective row, prompting me to pivot to improve the solution. However, after a few iterations, I encountered a state where the coefficients under the non-basic variables in the objective row were non-negative, but the pivot column suggested by the algorithm led to no positive entries in that column for the ratio test. This situation typically indicates that the problem is unbounded, meaning the objective function can be decreased indefinitely. The algorithm I followed couldn't find a bounded optimal solution under these conditions, so it concluded that the problem was unbounded (altough I'm really not sure about this output, at first the algorithm was in an infinite loop)

**final try without displaying the results**

In [18]:
from ortools.linear_solver import pywraplp

def solve_dual_problem():
    # Create the linear solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        print('SCIP solver is not available.')
        return

    # Define variables for the dual problem
    # These will correspond to the constraints in the primal problem
    y1 = solver.NumVar(0, solver.infinity(), 'y1')
    y2 = solver.NumVar(0, solver.infinity(), 'y2')
    y3 = solver.NumVar(0, solver.infinity(), 'y3')

    # Objective function: Minimize 145*y1 + 260*y2 + 190*y3
    objective = solver.Objective()
    objective.SetCoefficient(y1, 145)
    objective.SetCoefficient(y2, 260)
    objective.SetCoefficient(y3, 190)
    objective.SetMinimization()

    # Constraint 1: 5*y1 + 4*y2 + y3 >= 1
    c1 = solver.Constraint(1, solver.infinity())
    c1.SetCoefficient(y1, 5)
    c1.SetCoefficient(y2, 4)
    c1.SetCoefficient(y3, 1)

    # Constraint 2: 2*y1 + 8*y2 + y3 >= 4
    c2 = solver.Constraint(4, solver.infinity())
    c2.SetCoefficient(y1, 2)
    c2.SetCoefficient(y2, 8)
    c2.SetCoefficient(y3, 1)

    # Constraint 3: 2*y1 - 8*y2 + 4*y3 >= 2
    c3 = solver.Constraint(2, solver.infinity())
    c3.SetCoefficient(y1, 2)
    c3.SetCoefficient(y2, -8)
    c3.SetCoefficient(y3, 4)

    # Solve the problem and print the results
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        print('Optimal Solution Found:')
        print('y1 =', y1.solution_value())
        print('y2 =', y2.solution_value())
        print('y3 =', y3.solution_value())
        print('Objective value =', objective.Value())
    elif status == pywraplp.Solver.INFEASIBLE:
        print('No feasible solution found. The problem is infeasible.')
    elif status == pywraplp.Solver.UNBOUNDED:
        print('The problem is unbounded.')
    else:
        print('Solver status is', status)

# Run the dual problem solver
solve_dual_problem()


Optimal Solution Found:
y1 = 1.5
y2 = 0.12499999999999994
y3 = 0.0
Objective value = 250.00000000000003


Using the OR-Tools solver, I found an optimal solution for the dual linear programming problem. The dual variables \( y1 \), \( y2 \), and \( y3 \) had values of 1.5, approximately 0.125, and 0.0, respectively, with the minimized objective value being close to 250. This means that to minimize the cost, given the constraints, we should allocate resources in such a way that \( y1 \) is 1.5, \( y2 \) is 0.125, and \( y3 \) is not used. This allocation results in the lowest possible cost of 250 while still satisfying all constraints. The solver's output verifies the feasibility and boundedness of the problem, contrasting earlier tableau iterations that suggested the problem was unbounded.

**4. Is $Q$ the (optimal) solution for the primal problem ?**

 Since duality theory tells us that the optimal values of the primal and dual problems are equal under certain conditions, and the dual problem's optimal objective value is 250, if $Q$ is indeed optimal for the primal, then substituting $Q$ into the primal's objective function should yield the same value of 250.

In [22]:
# Define the primal objective function coefficients
primal_obj_coeffs = [1, 4, 2]

# Define the solution Q for the primal problem
Q = [0, 52.5, 20]

# Calculate the primal objective value for Q
primal_obj_value = sum(coef * Q_i for coef, Q_i in zip(primal_obj_coeffs, Q))

# Output the result in Markdown AD style
primal_solution_statement = f"""
After running the OR-Tools solver to find the optimal dual variables and their corresponding objective value, I used the primal variable values \( Q=(x_1,x_2,x_3)=(0,52.5,20) \) provided in the question to calculate the objective value for the primal problem.

By substituting \( Q \) into the primal's objective function, I found the objective value to be {primal_obj_value:.2f}. To verify if \( Q \) is the optimal solution for the primal problem, this calculated objective value must be equal to the dual problem's optimal objective value, which is 250.00.

Therefore, I conclude that:
"""

# Check if the primal objective value for Q is equal to the dual's optimal objective value (250)
if primal_obj_value == 250:
    primal_solution_statement += "Yes, \( Q \) is the optimal solution for the primal problem."
else:
    primal_solution_statement += "No, \( Q \) is not the optimal solution for the primal problem, as the calculated objective value does not match the dual's optimal value."

print(primal_solution_statement)



After running the OR-Tools solver to find the optimal dual variables and their corresponding objective value, I used the primal variable values \( Q=(x_1,x_2,x_3)=(0,52.5,20) \) provided in the question to calculate the objective value for the primal problem.

By substituting \( Q \) into the primal's objective function, I found the objective value to be 250.00. To verify if \( Q \) is the optimal solution for the primal problem, this calculated objective value must be equal to the dual problem's optimal objective value, which is 250.00.

Therefore, I conclude that:
Yes, \( Q \) is the optimal solution for the primal problem.


<>:17: SyntaxWarning: invalid escape sequence '\('
<>:17: SyntaxWarning: invalid escape sequence '\('
<>:21: SyntaxWarning: invalid escape sequence '\('
<>:23: SyntaxWarning: invalid escape sequence '\('
<>:17: SyntaxWarning: invalid escape sequence '\('
<>:17: SyntaxWarning: invalid escape sequence '\('
<>:21: SyntaxWarning: invalid escape sequence '\('
<>:23: SyntaxWarning: invalid escape sequence '\('
C:\Users\abdou\AppData\Local\Temp\ipykernel_2836\3169543394.py:17: SyntaxWarning: invalid escape sequence '\('
  """
C:\Users\abdou\AppData\Local\Temp\ipykernel_2836\3169543394.py:17: SyntaxWarning: invalid escape sequence '\('
  """
C:\Users\abdou\AppData\Local\Temp\ipykernel_2836\3169543394.py:21: SyntaxWarning: invalid escape sequence '\('
  primal_solution_statement += "Yes, \( Q \) is the optimal solution for the primal problem."
C:\Users\abdou\AppData\Local\Temp\ipykernel_2836\3169543394.py:23: SyntaxWarning: invalid escape sequence '\('
  primal_solution_statement += "No, \( Q \